Packages

In [1]:
import nltk
from nltk import bigrams, trigrams
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import ConditionalFreqDist

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("c:\\Users\\s2421127\\Documents\\NLP Project\\ObuayaO\\NLP project\\02.09.24_ns.csv", encoding = 'unicode_escape')

In [6]:
df.head()

,EudraCT_No,Title,Phase,Objective,End_date,Sample_size,pr_endpoint,endpoint_description,Treatment,LT_followup,manual_label,Reasoning,bigrams,trigrams,pr_bigrams,pr_trigrams
0,2018-003243-39,"A Phase 3, Randomized, Double-Blind, Placebo-C...",3,The primary purpose of this study is to evalua...,NaN,175,Primary: Change From Baseline In Myasthenia Gr...,MG-ADL: 8-point questionnaire focusing on rele...,Ravulizumab,No,0,Specific to disease HRQoL questionnaire,activating notch,activating notch mutations,amifampridine phosphate,exam digitally video
1,2009-016138-29,"âRandomized, Multicenter, Open-label, Phase ...",iii,To compare the efficacy of plitidepsin in comb...,20-Nov-17,255,Primary: Progression-free Survival (Independen...,The primary study analysis was based on extern...,Aplidin,No,2,Standard surrogate outcome,angle resolution,bleed hepatic encephalopathy,consciousness awareness,delayed cerebral ischemia
2,2016-000474-38,"A Multicenter, 2-Cohort Trial to First Assess ...",0,To demonstrate that fenfluramine hydrochloride...,05-Jun-18,87,Primary: Change in Convulsive Seizure Frequenc...,Baseline-adjusted in CSF (mean number of convu...,fenfluramine hydrochloride,Yes,0,Development of disease,anticoagulant apixaban,boundary shift integral,cover orofacial,microglia activation via
3,2014-000418-75,"A Multicenter, Multinational, Randomized, Doub...",0,The primary objective of this study was to ass...,19-Jun-18,352,Primary: Change From Baseline in UHDRS-TMS at ...,"UHDRS assess motor function, cognition, behavi...",Placebo,No,0,Specific to disease HRQoL questionnaire,bacterial peritonitis,exam digitally video,deep inspiration,positron emission tomography
4,2012-002933-12,A Phase II pilot study to explore treatment wi...,ii,To determine whether patients taking a medicin...,10-Dec-18,8,"Primary: Workload',",All participants cycled on a cycle ergometer. ...,Sodium Valproate,No,1,Endpoint on function,biliary cholangitis,meld uncontrolled ascites,detect follow,well extremity truncal


Slope plot

In [7]:
pfo_df = df[df['manual_label'] == 0]
io_df = df[df['manual_label']== 1]
so_df = df[df['manual_label']== 2]

In [11]:
def rel_freq(x):
    freqs = [(value, x.count(value) / len(x)) for value in set(x)] 
    return freqs

In [12]:
pfo_df['corpus'] = pfo_df['pr_endpoint'] + " " + pfo_df['endpoint_description']

C:\Users\s2421127\AppData\Local\Temp\ipykernel_15680\1739584776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pfo_df['corpus'] = pfo_df['pr_endpoint'] + " " + pfo_df['endpoint_description']


In [13]:
# tf_idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pfo_df['corpus']) # primary endpoint text and concatenated text
vectorizer.get_feature_names_out()

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
df['collocates'] = df['bigrams'] + " " + df['trigrams'] # I think this is okay but nothing else

vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
X_collocates = vectorizer.fit_transform(df['collocates'])

y_labels = df['manual_label']

mi_scores = mutual_info_classif(X_collocates, manual_labels)

feature_names = vectorizer.get_feature_names_out()
mi_scores_df = pd.DataFrame({'Collocate': feature_names, 'MI Score': mi_scores})

mi_scores_df = mi_scores_df.sort_values(by='MI Score', ascending=False)
print(mi_scores_df.head(20))  # Show top 10 most informative bigrams/trigrams

bigrams - primary text

In [ ]:
X_collocates = vectorizer.fit_transform(df['pr_bigrams']) # This cannot be right

y_labels = df['manual_label']

mi_scores = mutual_info_classif(X_collocates, y_labels)

feature_names = vectorizer.get_feature_names_out()
mi_scores_df = pd.DataFrame({'Collocate': feature_names, 'MI Score': mi_scores})

mi_scores_df = mi_scores_df.sort_values(by='MI Score', ascending=False)
print(mi_scores_df.head(20))  # Show top 10 most informative bigrams/trigrams

Slope Plot

In [ ]:
# Calculate relative frequency 

In [ ]:
# Set a list of collocates that change the most 
collocates = [] # Do one for unigram, bigrams, trigrams

#plot the chart
fig, ax = plt.subplots()
for i in collocates:
    temp = df[df['country_origin'] == i]
    plt.plot(temp.year, temp.refugees_number, color='#0072BC', marker='o', markersize=5)
    # start label
    plt.text(temp.year.values[0]-0.4, temp.refugees_number.values[0], i, ha='right', va='center')
    # end label
    plt.text(temp.year.values[1]+0.4, temp.refugees_number.values[1], i, ha='left', va='center')
    
ticks = plt.xticks([2000, 2021])
xl = plt.xlim(1990,2031)
yl = plt.ylim(0, 7*1e6)

#set chart title
ax.set_title('Evolution of refugee population by country of origin |2000 vs 2021')

#set y-axis label
ax.set_ylabel('Number of people (millions)')

#format x-axis tick labels
def number_formatter(x, pos):
    if x >= 1e6:
        s = '{:1.0f}M'.format(x*1e-6)
    elif x < 1e6 and x > 0:
        s = '{:1.0f}K'.format(x*1e-3)
    else: 
        s = '{:1.0f}'.format(x)
    return s
ax.yaxis.set_major_formatter(number_formatter)

#set chart source and copyright
plt.annotate('Source: UNHCR Refugee Data Finder', (0,0), (0, -25), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
plt.annotate('©UNHCR, The UN Refugee Agency', (0,0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)

#adjust chart margin and layout
fig.tight_layout()

#show chart

In [6]:
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt

In [ ]:
collocate_features = pd.DataFrame(X_collocates.toarray(), columns=vectorizer.get_feature_names_out())

collocate_features['label'] = df['manual_label']

subset = collocate_features.sample(20, axis=1) 
subset['label'] = df['manual_label']


plt.figure(figsize=(10,6))
parallel_coordinates(subset, class_column='label', colormap=plt.get_cmap('Set3'))

plt.title("Parallel Coordinate Plot for Primary Text Bigram Collocates and Manual Labels")
plt.ylabel("Feature Value (TF-IDF Score)")
plt.xticks(rotation=45, fontsize=6, fontweight='bold')
plt.xlabel("Collocate Features")
plt.show()

In [ ]:
collocate_features = pd.DataFrame(X_collocates.toarray(), columns=vectorizer.get_feature_names_out())

collocate_features['label'] = df['manual_label']

subset = collocate_features.sample(20, axis=1) 
subset['label'] = df['manual_label']


plt.figure(figsize=(10,6))
parallel_coordinates(subset, class_column='label', colormap=plt.get_cmap('Set3'))

# Step 6: Display the plot
plt.title("Parallel Coordinate Plot for Collocates and Manual Labels")
plt.ylabel("Feature Value (TF-IDF Score)")
plt.xticks(rotation=45, fontsize=8, fontweight='bold')
plt.xlabel("Collocate Features")
plt.show()

Conditional frequency distribution

In [ ]:
collocates = vectorizer.inverse_transform(X_collocates)

cfdist = ConditionalFreqDist()

for collocate_list, label in zip(collocates, df['manual_label']):  
    collocate_str = ' '.join(collocate_list)  
    tokens = word_tokenize(collocate_str) 
    
    
    for word in tokens:
        cfdist[label][word] += 1

print(cfdist)


In [ ]:
collocates = vectorizer.inverse_transform(X_collocates)

cfdist = ConditionalFreqDist()

for collocate_list, label in zip(collocates, df['manual_label']):  
    collocate_str = ' '.join(collocate_list)  
    tokens = word_tokenize(collocate_str) 
    
    
    for ngram in tokens:
        cfdist[label][ngram] += 1

print(cfdist)

In [ ]:
for label in df['manual_label'].unique():
    print(f"\nMost common collocates for label {label}:")
    print(cfdist[label].most_common(10))

In [ ]:
bigram_cfdist = ConditionalFreqDist()
trigram_cfdist = ConditionalFreqDist()

for collocate_list, label in zip(collocates, df['manual_label']): 
    collocate_str = ' '.join(collocate_list)  
    tokens = word_tokenize(collocate_str)  

    
    bigram_list = list(bigrams(tokens))   
    trigram_list = list(trigrams(tokens))  
    
    for bigram in bigram_list:
        bigram_cfdist[label][bigram] += 1 

    for trigram in trigram_list:
        trigram_cfdist[label][trigram] += 1  


print("Most common bigrams by label:")
for label in df['manual_label'].unique():
    print(f"\nLabel {label}:")
    print(bigram_cfdist[label].most_common(10))  

print("\nMost common trigrams by label:")
for label in df['manual_label'].unique():
    print(f"\nLabel {label}:")
    print(trigram_cfdist[label].most_common(10))  


Primary endpoint text

In [30]:
def perform_collocation_analysis(file_path, text_columns): 
    """Perform collocation analysis on a given text file to find top bigrams and trigrams, excluding those that occur only once."""
    df = pd.read_csv("c:\\Users\\s2421127\\Documents\\NLP Project\\ObuayaO\\NLP project\\02.09.24_ns.csv", encoding = 'unicode_escape')
    text = " ".join(df[text_columns].astype(str).apply(lambda row: ' '.join(row), axis=1).tolist())

    tokens = word_tokenize(text)

    words = [word.lower() for word in tokens if word.isalpha()]
    
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    bigram_measures = BigramAssocMeasures()
    trigram_measures = TrigramAssocMeasures()
    bigram_finder = BigramCollocationFinder.from_words(filtered_words)
    trigram_finder = TrigramCollocationFinder.from_words(filtered_words)

    bigram_finder.apply_freq_filter(2)
    trigram_finder.apply_freq_filter(2)

    top_bigrams = bigram_finder.nbest(bigram_measures.pmi, 190) # Matched with how many rows I have
    top_trigrams = trigram_finder.nbest(trigram_measures.pmi, 190)

    return top_bigrams, top_trigrams

top_bigrams, top_trigrams = perform_collocation_analysis("c:\\Users\\s2421127\\Documents\\NLP Project\\ObuayaO\\NLP project\\02.09.24_ns.csv", ["pr_endpoint", "endpoint_description"])

In [ ]:
with open("collocation-results_pr.txt", "w", encoding='utf-8') as output_file:
    output_file.write("Top Bigram Collocations (excluding those occurring only once):\n")
    for bigram in top_bigrams:
        output_file.write(f"{bigram[0]} {bigram[1]}\n")
    output_file.write("\nTop Trigram Collocations (excluding those occurring only once):\n")
    for trigram in top_trigrams:
        output_file.write(f"{trigram[0]} {trigram[1]} {trigram[2]}\n")

print("Collocation analysis is complete. Results are saved in 'collocation-results_pr.txt'.")

In [ ]:
def co_occurrence(text, window_size=2):
    # Load stopwords
    stop_words = set(stopwords.words('english'))
    
    # Tokenize the text and filter out stopwords
    tokens = [token.lower() for token in word_tokenize(text) if token.isalpha() and token.lower() not in stop_words]
    
    # Initialize co-occurrence count
    co_occurrence_counts = Counter()
    
    # Calculate co-occurrences within the specified window size
    for i in range(len(tokens)):
        token = tokens[i]
        start = max(0, i - window_size)
        end = min(len(tokens), i + window_size + 1)
        for j in range(start, end):
            if i != j:
                co_occurred_token = tokens[j]
                co_occurrence_counts[(token, co_occurred_token)] += 1
    
    # Return the top 10 most common co-occurrences
    return co_occurrence_counts.most_common(10)

# Read the text file
with open("collocation-results_pr.txt", "r", encoding='utf-8') as file:
    text = file.read()

# Calculate the top 10 co-occurrences
top_co_occurrences = co_occurrence(text, window_size=2)

# Output the results
with open("top-co-occurrence-results_pr.txt", "w", encoding='utf-8') as output_file:
    for pair, freq in top_co_occurrences:
        output_file.write(f"{pair[0]}, {pair[1]}: {freq}\n")

print("Top 10 co-occurrence analysis is complete. Results are saved in 'top-co-occurrence-primary-results_190.txt'.")

In [ ]:
df['collocates'] = df['pr_bigrams'] + " " + df['pr_trigrams']

vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
X_collocates = vectorizer.fit_transform(df['collocates'])

y_labels = df['manual_label']

mi_scores = mutual_info_classif(X_collocates, manual_labels)

feature_names = vectorizer.get_feature_names_out()
mi_scores_df = pd.DataFrame({'Collocate': feature_names, 'MI Score': mi_scores})

mi_scores_df = mi_scores_df.sort_values(by='MI Score', ascending=False)
print(mi_scores_df.head(20))  # Show top 10 most informative bigrams/trigrams

In [ ]:
collocate_features = pd.DataFrame(X_collocates.toarray(), columns=vectorizer.get_feature_names_out())

collocate_features['label'] = df['manual_label']

subset = collocate_features.sample(20, axis=1) 
subset['label'] = df['manual_label']


plt.figure(figsize=(10,6))
parallel_coordinates(subset, class_column='label', colormap=plt.get_cmap('Set2'))

# Step 6: Display the plot
plt.title("Parallel Coordinate Plot for Primary Text Collocates and Manual Labels")
plt.ylabel("Feature Value (TF-IDF Score)")
plt.xticks(rotation=45, fontsize=6, fontweight='bold')
plt.xlabel("Collocate Features")
plt.show()

In [44]:
collocates = vectorizer.inverse_transform(X_collocates)

cfdist = ConditionalFreqDist()

for collocate_list, label in zip(collocates, df['manual_label']):  
    collocate_str = ' '.join(collocate_list)  
    tokens = word_tokenize(collocate_str) 
    
    
    for word in tokens:
        cfdist[label][word] += 1

In [ ]:
bigram_cfdist = ConditionalFreqDist()
trigram_cfdist = ConditionalFreqDist()

for collocate_list, label in zip(collocates, df['manual_label']): 
    collocate_str = ' '.join(collocate_list)  
    tokens = word_tokenize(collocate_str)  

    
    bigram_list = list(bigrams(tokens))   
    trigram_list = list(trigrams(tokens))  
    
    for bigram in bigram_list:
        bigram_cfdist[label][bigram] += 1 

    for trigram in trigram_list:
        trigram_cfdist[label][trigram] += 1  


print("Most common bigrams by label:")
for label in df['manual_label'].unique():
    print(f"\nLabel {label}:")
    print(bigram_cfdist[label].most_common(10))  

print("\nMost common trigrams by label:")
for label in df['manual_label'].unique():
    print(f"\nLabel {label}:")
    print(trigram_cfdist[label].most_common(10))  


LOOCV

In [7]:
import numpy as np
import seaborn as sns

In [8]:
import re

In [9]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.lower()  # Convert to lower case
    text = text.split()  # Split into words
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]  # Lemmatize and remove stopwords # Get rid of primary as a stop word
    return ' '.join(text)

In [11]:
import unicodedata
def strip_accents(text):
    return ''. join(word for word in unicodedata.normalize ('NFD', text)
                     if unicodedata.category(word) != 'Mn')

In [12]:
df['Title'] = df['Title'].apply(preprocess_text)
df['Objective'] = df['Objective'].apply(preprocess_text)
df['pr_endpoint'] = df['pr_endpoint'].apply(preprocess_text)

In [13]:
df['endpoint_description'] = df['endpoint_description'].apply(str)

In [14]:
df['endpoint_description'] = df['endpoint_description'].apply(preprocess_text)

In [15]:
df['Title'] = df['Title'].apply(strip_accents)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
text_columns = ['Title', 'Objective', 'pr_endpoint', 'endpoint_description', 'LT_followup']
X = df[text_columns] # What are the predictors of primary endpoint type? I do not know if the n-grams should be included here as they are not 'natural' observations [ 'bigrams', 'trigrams', 'pr_bigrams', 'pr_trigrams']
y = df['manual_label'].values

In [18]:
X_combined = X[text_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [19]:
tfidf = TfidfVectorizer(preprocessor=preprocess_text, strip_accents='unicode')

In [20]:

X_tfidf = tfidf.fit_transform(X_combined)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [23]:
# Might now be obsolete
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', tfidf, text_columns)  # Apply TF-IDF to the text columns
    ]
)

In [24]:
# Might delete
pipeline = make_pipeline(preprocessor, RandomForestRegressor(n_estimators = 50, random_state = 4)) # 50 to 500 is the standard 

In [23]:
rfc = RandomForestClassifier(n_estimators = 50, random_state = 4)

In [24]:
from sklearn.model_selection import cross_val_score
# Set the number of folds equal to the number of observations

In [25]:
from sklearn.metrics import mean_absolute_error, make_scorer # MAE is better for interpretability 
mae_scorer = make_scorer(mean_absolute_error)
#import recall, precision, cm

In [26]:
n = X.shape[0]

In [27]:
cv_results = cross_val_score(
    estimator=rfc, # Changed from pipeline
    X=X_tfidf, # changed from X to X_tfidf
    y=y,
    cv=n,  
    scoring=mae_scorer
)

Debugging

In [ ]:
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

In [ ]:
print(df.isnull().sum())

In [ ]:
empty_rows = X[text_columns].apply(lambda x: x.str.strip()).eq('').any(axis=1)
print(f"Rows with empty or all-whitespace text: {empty_rows.sum()}")
print(df[empty_rows])  # Print the rows with empty or whitespace text


In [ ]:
print(cv_results)
print('The mean: {}'.format(cv_results.mean()))
print('The std: {}'.format(cv_results.std()))